# Exploratory Data Analysis - Part 1

### In this notebook, the data from Iowa Housing Price Dataset will be examined to clean any obvious extraneous values before visual exploration (in Part 2)

Let's first import the data from the training .csv file into a dataframe:

In [1]:
# Import tidyverse library for data analysis
library(tidyverse)

# Data file was already downloaded and saved locally
train <- read_csv("train.csv",show_col_types = FALSE)

-- Attaching packages ------------------------------------------------------------------------------- tidyverse 1.3.1 --

v ggplot2 3.3.5     v purrr   0.3.4
v tibble  3.1.6     v dplyr   1.0.8
v tidyr   1.2.0     v stringr 1.4.0
v readr   2.1.2     v forcats 0.5.1

-- Conflicts ---------------------------------------------------------------------------------- tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()



Take a quick look at the data:

In [2]:
head(train)
glimpse(train)

Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,...,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>
1,60,RL,65,8450,Pave,NA,Reg,Lvl,AllPub,...,0,NA,NA,NA,0,2,2008,WD,Normal,208500
2,20,RL,80,9600,Pave,NA,Reg,Lvl,AllPub,...,0,NA,NA,NA,0,5,2007,WD,Normal,181500
3,60,RL,68,11250,Pave,NA,IR1,Lvl,AllPub,...,0,NA,NA,NA,0,9,2008,WD,Normal,223500
4,70,RL,60,9550,Pave,NA,IR1,Lvl,AllPub,...,0,NA,NA,NA,0,2,2006,WD,Abnorml,140000
5,60,RL,84,14260,Pave,NA,IR1,Lvl,AllPub,...,0,NA,NA,NA,0,12,2008,WD,Normal,250000
6,50,RL,85,14115,Pave,NA,IR1,Lvl,AllPub,...,0,NA,MnPrv,Shed,700,10,2009,WD,Normal,143000


Rows: 1,460
Columns: 81
$ Id            <dbl> 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 1~
$ MSSubClass    <dbl> 60, 20, 60, 70, 60, 50, 20, 60, 50, 190, 20, 60, 20, 20,~
$ MSZoning      <chr> "RL", "RL", "RL", "RL", "RL", "RL", "RL", "RL", "RM", "R~
$ LotFrontage   <dbl> 65, 80, 68, 60, 84, 85, 75, NA, 51, 50, 70, 85, NA, 91, ~
$ LotArea       <dbl> 8450, 9600, 11250, 9550, 14260, 14115, 10084, 10382, 612~
$ Street        <chr> "Pave", "Pave", "Pave", "Pave", "Pave", "Pave", "Pave", ~
$ Alley         <chr> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, ~
$ LotShape      <chr> "Reg", "Reg", "IR1", "IR1", "IR1", "IR1", "Reg", "IR1", ~
$ LandContour   <chr> "Lvl", "Lvl", "Lvl", "Lvl", "Lvl", "Lvl", "Lvl", "Lvl", ~
$ Utilities     <chr> "AllPub", "AllPub", "AllPub", "AllPub", "AllPub", "AllPu~
$ LotConfig     <chr> "Inside", "FR2", "Inside", "Corner", "FR2", "Inside", "I~
$ LandSlope     <chr> "Gtl", "Gtl", "Gtl", "Gtl", "Gtl", "Gtl", "Gtl", "Gtl", ~
$ Neighborhood  

That's quite a few columns with categorical variables and it's clear some have "NA" values.

Let's see which columns contain NA:

In [3]:
NA_columns <- names(train)[sapply(train,anyNA)]
NA_columns

[1] "LotFrontage"  "Alley"        "MasVnrType"   "MasVnrArea"   "BsmtQual"    
 [6] "BsmtCond"     "BsmtExposure" "BsmtFinType1" "BsmtFinType2" "Electrical"  
[11] "FireplaceQu"  "GarageType"   "GarageYrBlt"  "GarageFinish" "GarageQual"  
[16] "GarageCond"   "PoolQC"       "Fence"        "MiscFeature"

19 columns out of 80 data columns.


The documentation appears to explain most of these:

LotFrontage - a house lacking any street frontage seems odd

Alley - no alley present

MasVnrType, MasVnrArea - Refer to masonry vaneer, so NA is not surprising in its absence, although the documentation lists "None" as an option not "NA"; may need to investigate further

BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinType2 - no basement present

Electrical - since there are no NA's in the Utilities column, and all options in that column imply at minimum the house has electricity, it seems odd the Electrical column contains NA's

FireplaceQu - no fireplace present

GarageType, GarageYrBlt, GarageFinish, GarageQual, GarageCond - no garage present

PoolQC - no pool present

Fence - no fence present

MiscFeature - no extra features

So LotFrontage and Electrical are worth checking to see why there are NA's present, and MasVnrType/MasVnrArea should also be looked at to see if there is a difference between NA and None.

In [4]:
train %>% summarize(across(c('LotFrontage','Electrical','MasVnrType','MasVnrArea'), ~ sum(is.na(.))))

LotFrontage,Electrical,MasVnrType,MasVnrArea
<int>,<int>,<int>,<int>
259,1,8,8


The counts in the Electrical & two masonry columns look fairly low - this is probably just missing / incorrectly entered data. However, that is a lot of houses that have "NA" for LotFrontage (259 out of 1460). Let's see if there is anything obvious these could correspond to see if they can be easily replaced with an actual value.

How about zoning?

In [5]:
train %>% filter(is.na(LotFrontage)) %>% count(MSZoning)

MSZoning,n
<chr>,<int>
FV,8
RH,3
RL,229
RM,19


RL is residential low density. Low density house should intuitively have more frontage than high density, so these NA's probably don't correspond to 0.
This distribution also roughly reflects the overall zoning distribution:

In [6]:
train %>% count(MSZoning)

MSZoning,n
<chr>,<int>
C (all),10
FV,65
RH,16
RL,1151
RM,218


What about lot shape?

In [7]:
cat('LotFrontage NA summary')
train %>% filter(is.na(LotFrontage)) %>% count(LotShape)
cat('\n','Overall summary')
train %>% count(LotShape)

LotFrontage NA summary

LotShape,n
<chr>,<int>
IR1,167
IR2,15
IR3,3
Reg,74



 Overall summary

LotShape,n
<chr>,<int>
IR1,484
IR2,41
IR3,10
Reg,925


Pretty even mix there (a bit more IR1 irregular lots have NA than regular lots if comparing to the overall summary, but still not conclusive). Maybe lot configuration is relevant.

In [8]:
cat('LotFrontage NA summary')
train %>% filter(is.na(LotFrontage)) %>% count(LotConfig)
cat('\n','Overall summary')
train %>% count(LotConfig)

LotFrontage NA summary

LotConfig,n
<chr>,<int>
Corner,62
CulDSac,49
FR2,14
Inside,134



 Overall summary

LotConfig,n
<chr>,<int>
Corner,263
CulDSac,94
FR2,47
FR3,4
Inside,1052


More Inside lots than not, but not definitive.

What about neighborhood? Maybe house location impacted the data.

In [9]:
cat('LotFrontage NA summary')
train %>% filter(is.na(LotFrontage)) %>% count(Neighborhood)
cat('\n','Overall summary')
train %>% count(Neighborhood)

LotFrontage NA summary

Neighborhood,n
<chr>,<int>
Blmngtn,3
BrkSide,7
ClearCr,15
CollgCr,24
Crawfor,10
Edwards,8
Gilbert,30
IDOTRR,3
MeadowV,2



 Overall summary

Neighborhood,n
<chr>,<int>
Blmngtn,17
Blueste,2
BrDale,16
BrkSide,58
ClearCr,28
CollgCr,150
Crawfor,51
Edwards,100
Gilbert,79


Nothing jumping out here. There is also a column for "proximity to various features," such as railroads. Let's see if this is the cause.

In [10]:
cat('LotFrontage NA summary')
train %>% filter(is.na(LotFrontage)) %>% count(Condition1)
cat('\n','Overall summary')
train %>% count(Condition1)

LotFrontage NA summary

Condition1,n
<chr>,<int>
Artery,2
Feedr,14
Norm,223
PosA,4
PosN,10
RRAe,2
RRAn,2
RRNe,1
RRNn,1



 Overall summary

Condition1,n
<chr>,<int>
Artery,48
Feedr,81
Norm,1260
PosA,8
PosN,19
RRAe,11
RRAn,26
RRNe,2
RRNn,5


Normal is the biggest category with lot frontage = NA. Building type is also a possibility (since townhouses as an option)

In [11]:
cat('LotFrontage NA summary')
train %>% filter(is.na(LotFrontage)) %>% count(BldgType)
cat('\n','Overall summary')
train %>% count(BldgType)

LotFrontage NA summary

BldgType,n
<chr>,<int>
1Fam,226
2fmCon,3
Duplex,5
Twnhs,3
TwnhsE,22



 Overall summary

BldgType,n
<chr>,<int>
1Fam,1220
2fmCon,31
Duplex,52
Twnhs,43
TwnhsE,114


Nope, mostly single family homes. Is there any correlation between lot area and the lot frontage NA?

In [12]:
cat('LotFrontage NA summary')
train %>% filter(is.na(LotFrontage)) %>% summarize(mean(LotArea), median(LotArea))
cat('\n','LotFrontage non - NA summary')
train %>% filter(!is.na(LotFrontage)) %>% summarize(mean(LotArea), median(LotArea))

LotFrontage NA summary

mean(LotArea),median(LotArea)
<dbl>,<dbl>
13137.37,10624



 LotFrontage non - NA summary

mean(LotArea),median(LotArea)
<dbl>,<dbl>
9951.699,9262


That's a little interesting. So the houses that are missing lot frontage data tend to be on larger lots. What about square footage of the house itself?

In [13]:
cat('LotFrontage NA summary')
train %>% filter(is.na(LotFrontage)) %>% summarize(mean(GrLivArea))
cat('\n','LotFrontage non-NA summary')
train %>% filter(!is.na(LotFrontage)) %>% summarize(mean(GrLivArea))

LotFrontage NA summary

mean(GrLivArea)
<dbl>
1535.548



 LotFrontage non-NA summary

mean(GrLivArea)
<dbl>
1511.132


Not much of difference there - and the lot area isn't much to go on on its own.

It seems a shame to drop ~18% of the data, but it doesn't seem like there is an obvious value to replace these NA values with.

Let check the two masonry columns quick before dropping NA data:

In [14]:
train %>% count(MasVnrType)
train %>% select(MasVnrType,MasVnrArea) %>% filter(is.na(MasVnrType))

MasVnrType,n
<chr>,<int>
BrkCmn,15
BrkFace,445
None,864
Stone,128
NA,8


MasVnrType,MasVnrArea
<chr>,<dbl>
NA,NA
NA,NA
NA,NA
NA,NA
NA,NA
NA,NA
NA,NA
NA,NA


So the houses with NA in the masonry column are only a small fraction of the houses that actually have masonry = "None". Just to confirm...

In [15]:
train %>% filter(MasVnrArea == 0) %>% count (MasVnrArea)

MasVnrArea,n
<dbl>,<int>
0,861


...the zero areas in MasVnrArea account for almost all (all but 3) of the "None" values in the MasVnrType column. The NA values could correspond to zero, but it is still prudent to remove them anyway, since this is uncertain.

One last thing to check is the impact of the LotFrontage values that are about to be dropped on the sales price:

In [16]:
train %>% filter(is.na(LotFrontage)) %>% summarize('Mean of SalePrice for LotFrontage NA' = mean(SalePrice), 'Median of SalePrice for LotFrontage NA' = median(SalePrice))
train %>% filter(!is.na(LotFrontage)) %>% summarize('Mean of SalePrice for remainder of houses' = mean(SalePrice), 'Median of SalePrice for remainder of houses' = median(SalePrice))

Mean of SalePrice for LotFrontage NA,Median of SalePrice for LotFrontage NA
<dbl>,<dbl>
181620.1,172400


Mean of SalePrice for remainder of houses,Median of SalePrice for remainder of houses
<dbl>,<dbl>
180770.5,159500


Hardly any difference in the mean, but there is a difference in the medians, suggesting these are higher value houses (intuition would also support this based on the larger LotArea, which would probably result in increased property values). However, without any definitive indication what is an appropriate replacement for these NA values in the LotFrontage column, they will have to be dropped.

(One possibility is that these are higher value houses that are displaced back from the streetfront, and NA in the LotFrontage is how this was recorded, but this is only a guess that has not been proven.)

In [17]:
train <- train %>% drop_na(LotFrontage, Electrical, MasVnrType, MasVnrArea)

In [18]:
NA_columns <- names(train)[sapply(train,anyNA)]
NA_columns

[1] "Alley"        "BsmtQual"     "BsmtCond"     "BsmtExposure" "BsmtFinType1"
 [6] "BsmtFinType2" "FireplaceQu"  "GarageType"   "GarageYrBlt"  "GarageFinish"
[11] "GarageQual"   "GarageCond"   "PoolQC"       "Fence"        "MiscFeature"

Only the columns where NA has a documented meaning remain.

Now that extraneous NA values have been removed, we can start visualizing the data to gain further insights (continued in Part 2).